In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import torch
import torch.nn as nn
import torchvision
import numpy as np
import matplotlib.pyplot as plt

In [2]:
if torch.cuda.is_available():
    device="cuda"
else:
    device="cpu"

In [3]:
training=torchvision.datasets.CIFAR10(
    root="data", 
    train=True, 
    download=True
)

In [4]:
images=[]
modified_images=[]
for example in training:
    images.append(np.array(example[0])[14:18, 14:18, :]/255)
    x=np.array(example[0])/255
    x[14:18, 14:18, :]=np.zeros((4, 4, 3))
    modified_images.append(x)

In [5]:
plt.imshow(modified_images[1711])

In [6]:
plt.imshow(images[1711])

In [7]:
batch_size=100
batch_number=50000//batch_size
image_batches=[]
modified_image_batches=[]
for batch in range(batch_number):
    x=min(batch_size, 50000-batch*batch_size)
    mini_batch=torch.zeros(x, 3, 4, 4).to(device)
    modified_mini_batch=torch.zeros(x, 3, 32, 32).to(device)
    for j in range(x):
        mini_batch[j]=torch.tensor(images[batch*batch_size+j], dtype=torch.float32).permute(2, 0, 1).to(device)
        modified_mini_batch[j]=torch.tensor(modified_images[batch*batch_size+j], dtype=torch.float32).permute(2, 0, 1).to(device)
    image_batches.append(mini_batch)
    modified_image_batches.append(modified_mini_batch)

In [8]:
modified_image_batches[11].shape

In [36]:
class Neural_Network(nn.Module):
    def __init__(self):
        super(Neural_Network, self).__init__()
        self.network=nn.Sequential(
            nn.Conv2d(3, 64, 3), 
            nn.ReLU(), 
            nn.Conv2d(64, 128, 3), 
            nn.ReLU(), 
            nn.Conv2d(128, 256, 3), 
            nn.ReLU(), 
            nn.Conv2d(256, 512, 3), 
            nn.ReLU(), 
            nn.MaxPool2d(2), 
            nn.Conv2d(512, 256, 3), 
            nn.ReLU(), 
            nn.Conv2d(256, 128, 3),
            nn.ReLU(), 
            nn.Conv2d(128, 64, 3), 
            nn.ReLU(), 
            nn.Conv2d(64, 3, 3)
        )
    def forward(self, x):
        return self.network(x)

In [37]:
model=Neural_Network().to(device)

In [38]:
loss_fn=nn.L1Loss()
optimizer=torch.optim.SGD(model.parameters(), lr=0.1)

In [47]:
epochs=10
data=[]
for epoch in range(epochs):
    for batch in range(batch_number):
        res=model(modified_image_batches[batch])
        loss=loss_fn(res, image_batches[batch])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if batch%50==0:
            distance=round(float(loss)*255, 2)
            print(f"{epoch} {batch} {distance}")
            data.append(distance)

In [48]:
plt.plot(data[:])

In [49]:
with torch.no_grad():
    z=np.array(model(modified_image_batches[179][3]).permute(1, 2, 0).detach().cpu())
    iota=np.array(modified_image_batches[179][3].permute(1, 2, 0).detach().cpu())
plt.subplot(1, 2, 1)
plt.imshow(z)
plt.subplot(1, 2, 2)
plt.imshow(iota)

In [50]:
iota[14:18, 14:18, :]=z
plt.imshow(iota)